# Fine-tune Dolly-v2-3b with Ray AIR LightningTrainer and FSDP

In [1]:
# TODO(yunxuan): remove it
import os
os.environ["RAY_ML_DEV"] = "1"

## Set up ray cluster 
In this example, we are using a ray cluster with 16 g4dn.4xlarge instances. Each instance has one Tesla T4 GPU (16GiB Memory). 

We define a `runtime_env` to install the necessary Python libraries on each node. You can skip this step if you have already installed all the required packages in your workers' base image.

In [2]:
import ray

ray.init(
    runtime_env={
        "pip": [
            "datasets",
            "evaluate",
            "transformers>=4.26.0",
            "torch>=1.12.0",
            "pytorch_lightning>=2.0",
        ]
    }
)

/home/ray/anaconda3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
find: ‘.git’: No such file or directory
2023-05-02 19:46:34,146	INFO worker.py:1432 -- Connecting to existing Ray cluster at address: 10.0.106.32:6379...
2023-05-02 19:46:34,164	INFO worker.py:1607 -- Connected to Ray cluster. View the dashboard at https://console.anyscale-staging.com/api/v2/sessions/ses_m411tiqu8eluvt1k5ivfqj4q5r/services?redirect_to=dashboard 
2023-05-02 19:46:34,739	INFO packaging.py:520 -- Creating a file package for local directory '/tmp/ray_tmp_module/ray'.
2023-05-02 19:46:34,841	WARNING packaging.py:394 -- File /tmp/ray_tmp_module/ray/jars/ray_dist.jar is very large (30.48MiB). Consider adding this file to the 'excludes' list to skip uploading it: `ray.init(..., runtime_env={'excludes': ['/tmp/ray_tmp_module/ray/

Python version:,3.8.13
Ray version:,3.0.0.dev0
Dashboard:,http://console.anyscale-staging.com/api/v2/sessions/ses_m411tiqu8eluvt1k5ivfqj4q5r/services?redirect_to=dashboard


In [3]:
num_workers = 16
batch_size_per_worker = 8
MODEL_NAME = "databricks/dolly-v2-3b"

## Prepare your data 
We are using tiny_shakespeare for fine-tuning, which contains 40,000 lines of Shakespeare from a variety of Shakespeare's plays. Featured in Andrej Karpathy's blog post ['The Unreasonable Effectiveness of Recurrent Neural Networks'](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). 

Dataset samples:
```
BAPTISTA:
I know him well: you are welcome for his sake.

GREMIO:
Saving your tale, Petruchio, I pray,
Let us, that are poor petitioners, speak too:
Baccare! you are marvellous forward.

PETRUCHIO:
O, pardon me, Signior Gremio; I would fain be doing.
```

Here, we have adopted similar pre-processing logic from another demo: {ref}`GPT-J-6B Fine-Tuning with Ray AIR and DeepSpeed <gpt-j-6b-finetune-deepspeed>`.

In [ ]:
import ray
import pandas as pd
from datasets import load_dataset
from ray.data.preprocessors import BatchMapper, Chain
from transformers import AutoTokenizer, AutoModelForCausalLM

def split_text(batch: pd.DataFrame) -> pd.DataFrame:
    text = list(batch["text"])
    flat_text = "".join(text)
    split_text = [
        x.strip()
        for x in flat_text.split("\n")
        if x.strip() and not x.strip()[-1] == ":"
    ]
    return pd.DataFrame(split_text, columns=["text"])


def tokenize(batch: pd.DataFrame) -> dict:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, padding_side="left")
    tokenizer.pad_token = tokenizer.eos_token
    ret = tokenizer(
        list(batch["text"]),
        truncation=True,
        max_length=256,
        padding="max_length",
        return_tensors="np",
    )
    ret["labels"] = ret["input_ids"].copy()
    return dict(ret)

splitter = BatchMapper(split_text, batch_format="pandas")
tokenizer = BatchMapper(tokenize, batch_format="pandas")
preprocessor = Chain(splitter, tokenizer)

hf_dataset = load_dataset("tiny_shakespeare")
ray_datasets = ray.data.from_huggingface(hf_dataset)

We first split the original paragraphs into multiple sentences, then tokenize them. Here are some samples:

In [12]:
ds = ray_datasets["train"]
splitter.fit_transform(ds).take(10)

2023-05-02 19:54:12,703	INFO streaming_executor.py:87 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[BatchMapper]
2023-05-02 19:54:12,704	INFO streaming_executor.py:88 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
2023-05-02 19:54:12,705	INFO streaming_executor.py:90 -- Tip: To enable per-operator progress reporting, set RAY_DATA_VERBOSE_PROGRESS=1.


[{'text': 'Before we proceed any further, hear me speak.'},
 {'text': 'Speak, speak.'},
 {'text': 'You are all resolved rather to die than to famish?'},
 {'text': 'Resolved. resolved.'},
 {'text': 'First, you know Caius Marcius is chief enemy to the people.'},
 {'text': "We know't, we know't."},
 {'text': "Let us kill him, and we'll have corn at our own price."},
 {'text': "Is't a verdict?"},
 {'text': "No more talking on't; let it be done: away, away!"},
 {'text': 'One word, good citizens.'}]

## Define your lightning model

In this example, we use the [Dolly-v2-3b](https://huggingface.co/databricks/dolly-v2-3b) model for finetuning. It is an instruction-following large language model trained on the Databricks machine learning platform that is licensed for commercial use. We load the model weights from Huggingface Model Hub and encapsulate it into a `pl.LightningModule`.

:::{note}
Make sure you pass the FSDP wrapped model parameters `self.trainer.model.parameters()` into the optimizer, instead of `self.model.parameters()`. 
:::


In [13]:
import torch
import pytorch_lightning as pl

class DollyV2Model(pl.LightningModule):
    def __init__(self, lr=2e-5, eps=1e-8):
        super().__init__()
        self.lr = lr
        self.eps = eps
        self.model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
        self.predictions = []
        self.references = []

    def forward(self, batch):
        outputs = self.model(
            batch["input_ids"], 
            attention_mask=batch["attention_mask"], 
            labels=batch["labels"]
        )
        loss = outputs[0]
        return loss

    def training_step(self, batch, batch_idx):
        loss = self.forward(batch)
        self.log("train_loss", loss, prog_bar=True, on_step=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.AdamW(self.trainer.model.parameters(), lr=self.lr, eps=self.eps)

## Configure your FSDP strategy
As Dolly-v2-3b is a relatively large model, it cannot be properly fit into a single commercial GPU. In this example, we use the FSDP strategy to shard model parameters across multiple workers. This allows us to avoid GPU out-of-memory issues and support a larger global batch size.

:::{note}
FSDP is a type of data parallelism that shards model parameters, optimizer states and gradients across DDP ranks. This was inspired by Xu et al. as well as the ZeRO Stage 3 from DeepSpeed. You may refer to these blogs for more information:

- [Getting Started with Fully Sharded Data Parallel(FSDP)](https://pytorch.org/tutorials/intermediate/FSDP_tutorial.html#:~:text=FSDP%20is%20a%20type%20of,sizes%20for%20our%20training%20job.)
- [Fully Sharded Data Parallel: faster AI training with fewer GPUs](https://engineering.fb.com/2021/07/15/open-source/fsdp/)
- [PyTorch FSDP Tutorial](https://www.youtube.com/watch?v=8_k76AHu__s&list=PL_lsbAsL_o2BT6aerEKgIoufVD_fodnuT)
:::

To start trainig with Lightning's [FSDPStrategy](https://lightning.ai/docs/pytorch/stable/api/lightning.pytorch.strategies.FSDPStrategy.html#lightning.pytorch.strategies.FSDPStrategy), you only need to provide the initialization arguments in `LightningConfigBuilder.strategy()`. Behind the scenes, LightningTrainer handles the cluster environment settings and job launching.


:::{tips}
Some tips for FSDP configutarion:
- `sharding_strategy`:
    - `ShardingStrategy.NO_SHARD`: Parameters, gradients, and optimizer states are not sharded. Similar to DDP.
    - `ShardingStrategy.SHARD_GRAD_OP`: Gradients and optimizer states are sharded during computation, and additionally, parameters are sharded outside computation.
    - `ShardingStrategy.FULL_SHARD`: Parameters, gradients, and optimizer states are sharded. It has minimal GRAM usage among the 3 options.
- `auto_wrap_policy`:
    - Model layers are often wrapped with FSDP in a layered fashion. This means that only the layers in a single FSDP instance are required to aggregate all parameters to a single device during forwarding or backward calculations.
    - Use `transformer_auto_wrap_policy` to automatically wrap each Transformer Block into a single FSDP instance. 
- `backward_prefetch` and `forward_prefetch`:
    - Overlap the upcoming all-gather while executing the current forward/backward pass. It can improve throughput but may slightly increase peak memory usage.
:::

In [14]:
import functools
from ray.train.lightning import LightningTrainer, LightningConfigBuilder
from ray.air.config import RunConfig, ScalingConfig, CheckpointConfig
from torch.distributed.fsdp.wrap import transformer_auto_wrap_policy
from torch.distributed.fsdp import ShardingStrategy, BackwardPrefetch
from transformers.models.gpt_neox.modeling_gpt_neox import GPTNeoXLayer

# Define the model sharding policy:
# Wrap every GPTNeoXLayer as its own FSDP instance
auto_wrap_policy = functools.partial(
    transformer_auto_wrap_policy,
    transformer_layer_cls = {GPTNeoXLayer}
)

# Aggregate all arguments for LightningTrainer
lightning_config = (
    LightningConfigBuilder()
    .module(cls=DollyV2Model, lr=2e-5, eps=1e-8)
    .trainer(
        max_epochs=1, 
        accelerator="gpu", 
        precision="16-mixed",
    )
    .strategy(
        name="fsdp",
        sharding_strategy=ShardingStrategy.FULL_SHARD,
        backward_prefetch=BackwardPrefetch.BACKWARD_PRE,
        forward_prefetch=True,
        auto_wrap_policy=auto_wrap_policy,
    )
    .checkpointing(save_last=True)
)

In [15]:
from pytorch_lightning.callbacks import TQDMProgressBar

# Create a customized progress bar for LightningTrainer
class DollyV2ProgressBar(TQDMProgressBar):
    def __init__(self, num_iters_per_epoch, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.num_iters_per_epoch = num_iters_per_epoch
    
    def on_train_epoch_start(self, trainer, *_):
        super().on_train_epoch_start(trainer, *_)
        self.train_progress_bar.reset(self.num_iters_per_epoch)

total_batches = splitter.fit_transform(ray_datasets["train"]).count()
num_iters_per_epoch = total_batches // (num_workers * batch_size_per_worker)
lightning_config.trainer(callbacks=[DollyV2ProgressBar(num_iters_per_epoch)])

2023-05-02 19:55:16,209	INFO streaming_executor.py:87 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[BatchMapper]
2023-05-02 19:55:16,210	INFO streaming_executor.py:88 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
2023-05-02 19:55:16,211	INFO streaming_executor.py:90 -- Tip: To enable per-operator progress reporting, set RAY_DATA_VERBOSE_PROGRESS=1.


## Fine-tune with LightningTrainer

```{note}
Here we upload the checkpoints to cloud storage by setting S3 bucket URI to {class}`air.RunConfig(storage_path) <ray.air.RunConfig>`. You can also write to your local file system. See {ref}`train-run-config` for an example.
```

In [16]:
# Save AIR checkpoints according to the performance on validation set
run_config = RunConfig(
    name="finetune_dolly-v2-3b",
    storage_path="s3://anyscale-staging-data-cld-kvedzwag2qa8i5bjxuevf5i7/ray-lightning-results/",
    checkpoint_config=CheckpointConfig(),
)

# Scale the DDP training workload across 16 GPUs
# You can change this config based on your compute resources.
scaling_config = ScalingConfig(
    num_workers=num_workers, use_gpu=True, resources_per_worker={"CPU": 12, "GPU": 1}
)

trainer = LightningTrainer(
    lightning_config=lightning_config.build(),
    run_config=run_config,
    scaling_config=scaling_config,
    datasets={"train": ray_datasets["train"]},
    datasets_iter_config={"batch_size": batch_size_per_worker},
    preprocessor=preprocessor,
)
result = trainer.fit()

result


2023-05-02 19:55:25,503	WARNING trial_runner.py:1607 -- The maximum number of pending trials has been automatically set to the number of available cluster CPUs, which is high (299 CPUs/pending trials). If you're running an experiment with a large number of trials, this could lead to scheduling overhead. In this case, consider setting the `TUNE_MAX_PENDING_TRIALS_PG` environment variable to the desired maximum number of concurrent trials.
(LightningTrainer pid=18441) 2023-05-02 19:55:32,963	INFO backend_executor.py:128 -- Starting distributed worker processes: ['18620 (10.0.106.32)', '6438 (10.0.94.249)', '6564 (10.0.114.130)', '6466 (10.0.64.74)', '6472 (10.0.117.249)', '26259 (10.0.103.148)', '6537 (10.0.99.42)', '6563 (10.0.83.176)', '6484 (10.0.88.86)', '6488 (10.0.98.35)', '6494 (10.0.86.75)', '26682 (10.0.91.32)', '6464 (10.0.106.11)', '26879 (10.0.119.134)', '6530 (10.0.105.226)', '25981 (10.0.123.24)']
(RayTrainWorker pid=18620) 2023-05-02 19:55:35,246	INFO config.py:86 -- Setti

Epoch 0:   0%|          | 0/168 [00:00<?, ?it/s]


(RayTrainWorker pid=18620) /home/ray/anaconda3/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
(RayTrainWorker pid=18620)   rank_zero_warn(


Epoch 0: 100%|██████████| 168/168 [21:21<00:00,  7.63s/it, v_num=0, train_loss=0.154]
Epoch 0: : 169it [21:29,  7.63s/it, v_num=0, train_loss=0.176]                       


Trial name,_report_on,date,done,epoch,experiment_tag,hostname,iterations_since_restore,node_ip,pid,should_checkpoint,step,time_since_restore,time_this_iter_s,time_total_s,timestamp,train_loss,training_iteration,trial_id
LightningTrainer_f41a9_00000,train_epoch_end,2023-05-02_20-20-04,True,0,0,ip-10-0-106-32,1,10.0.106.32,18441,True,169,1474.59,1474.59,1474.59,1683084004,0.175903,1,f41a9_00000


Epoch 0: : 169it [23:11,  8.24s/it, v_num=0, train_loss=0.176]


(RayTrainWorker pid=18620) `Trainer.fit` stopped: `max_epochs=1` reached.
(RayTrainWorker pid=18620) RayFSDPStrategy: tearing down strategy...
(LightningTrainer pid=18441) 2023-05-02 20:21:31,083	WARNING util.py:315 -- Uploading trial artifacts took 63.715 s, which may be a performance bottleneck. Consider saving fewer/smaller artifacts to the trial log directory, or disable artifact syncing with `SyncConfig(sync_artifacts=False)`.
2023-05-02 20:22:15,226	WARNING experiment_state.py:306 -- Syncing the experiment checkpoint to cloud took a long time with 41.43 seconds. This can be due to a large number of trials, large logfiles, or throttling from the remote storage provider for too frequent syncs. If your `CheckpointConfig.num_to_keep` is a low number, this can trigger frequent syncing, in which case you should increase it. 
2023-05-02 20:22:15,231	INFO tune.py:1010 -- Total run time: 1609.83 seconds (1568.28 seconds for the tuning loop).


Result(
  metrics={'_report_on': 'train_epoch_end', 'train_loss': 0.1759033203125, 'epoch': 0, 'step': 169, 'should_checkpoint': True, 'done': True, 'trial_id': 'f41a9_00000', 'experiment_tag': '0'},
  path='s3://anyscale-staging-data-cld-kvedzwag2qa8i5bjxuevf5i7/ray-lightning-results/finetune_dolly-v2-3b/LightningTrainer_f41a9_00000_0_2023-05-02_19-55-25',
  checkpoint=LightningCheckpoint(uri=s3://anyscale-staging-data-cld-kvedzwag2qa8i5bjxuevf5i7/ray-lightning-results/finetune_dolly-v2-3b/LightningTrainer_f41a9_00000_0_2023-05-02_19-55-25/checkpoint_000000)
)

In [17]:
checkpoint_uri = result.checkpoint.uri
checkpoint_local_dir = "/tmp/model-checkpoint"
# checkpoint_uri = "s3://yunxuanx-test/model-checkpoint/finetune-dolly-v2/LightningTrainer_18e2e_00000_0_2023-04-30_17-57-08/checkpoint_000000/"

os.makedirs(checkpoint_local_dir, exist_ok=True)
os.system(f"aws s3 sync {checkpoint_uri} {checkpoint_local_dir}")

download: s3://anyscale-staging-data-cld-kvedzwag2qa8i5bjxuevf5i7/ray-lightning-results/finetune_dolly-v2-3b/LightningTrainer_f41a9_00000_0_2023-05-02_19-55-25/checkpoint_000000/.metadata.pkl to ../../../../../../../../tmp/model-checkpoint/.metadata.pkl
download: s3://anyscale-staging-data-cld-kvedzwag2qa8i5bjxuevf5i7/ray-lightning-results/finetune_dolly-v2-3b/LightningTrainer_f41a9_00000_0_2023-05-02_19-55-25/checkpoint_000000/.is_checkpoint to ../../../../../../../../tmp/model-checkpoint/.is_checkpoint
download: s3://anyscale-staging-data-cld-kvedzwag2qa8i5bjxuevf5i7/ray-lightning-results/finetune_dolly-v2-3b/LightningTrainer_f41a9_00000_0_2023-05-02_19-55-25/checkpoint_000000/_preprocessor to ../../../../../../../../tmp/model-checkpoint/_preprocessor
download: s3://anyscale-staging-data-cld-kvedzwag2qa8i5bjxuevf5i7/ray-lightning-results/finetune_dolly-v2-3b/LightningTrainer_f41a9_00000_0_2023-05-02_19-55-25/checkpoint_000000/.tune_metadata to ../../../../../../../../tmp/model-checkp

0

## Text-generation with HuggingFace Pipeline

We can use the [HuggingFace Pipeline](https://huggingface.co/docs/transformers/main_classes/pipelines) to generate predictions from our fine-tuned model. Let's input some prompts and see if our tuned Dolly can speak like Shakespeare:

In [18]:
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, padding_side="right")
dolly = DollyV2Model.load_from_checkpoint(f"{checkpoint_local_dir}/model")

nlp_pipeline = pipeline(task="text-generation", model=dolly.model, tokenizer=tokenizer, device=0)

for prompt in ["This is", "I am", "Once more"]:
    print(nlp_pipeline(prompt, max_new_tokens=30, pad_token_id=tokenizer.eos_token_id))

[{'generated_text': 'This is the day that I was born, and this is the day that I shall die.'}]
[{'generated_text': 'I am a poor man, sir, and I am a soldier.'}]
[{'generated_text': 'Once more, my lord, I am your servant.'}]
